In [21]:

import pandas as pd
import csv
import numpy as np
import nltk
from nltk.stem import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
import seaborn as sb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import nltk
import nltk.corpus 
from nltk.tokenize import word_tokenize
from sklearn import svm
from sklearn.svm import SVC
from nltk.classify.scikitlearn import SklearnClassifier
import torch




df = pd.read_csv('liar_dataset\\filipino_dataset.csv')
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.8
train_news = df[msk]
test_news = df[~msk]
#data observation



def str2bool(v):
    if v.lower()  == 'true': return True
    elif v.lower() == 'false': return False
    else: raise argparse.ArgumentTypeError('Boolean value expected.')

def process_labels(labels):
    # If the task has multiple label columns
    if len(labels.shape) == 2:
        return torch.tensor(labels).float()

    # If the task has one label columns
    elif len(labels.shape) == 1:
        if type(labels[0]) is not int:
            labellist = list(set(labels))
            encodings = [labellist.index(l) for l in labels]
            return torch.tensor(encodings)
        return torch.tensor(labels)

def process_data(text, labels, tokenizer, msl=128):
    # Process text
    if len(text.shape) == 2: # Sentence pair tasks
        text = [tuple(t) for t in text]
        encodings = tokenizer(text, truncation='longest_first', padding='max_length', max_length=msl)
    elif len(text.shape) == 1: # Sentence tasks
        encodings = tokenizer(list(text), truncation='longest_first', padding='max_length', max_length=msl)

    # Process labels
    targets = process_labels(labels)

    # Convert to tensordataset
    if 'distilbert' in str(type(tokenizer)) or 'roberta' in str(type(tokenizer)):
        dataset = torch.utils.data.TensorDataset(torch.tensor(encodings.input_ids),
                                                torch.tensor(encodings.attention_mask), 
                                                targets)
    else:
        dataset = torch.utils.data.TensorDataset(torch.tensor(encodings.input_ids), 
                                                torch.tensor(encodings.token_type_ids), 
                                                torch.tensor(encodings.attention_mask), 
                                                targets)

    return dataset

#creating ngrams
#unigram 
def create_unigram(words):
    assert type(words) == list
    return words


def create_bigrams(words):
    assert type(words) == list
    skip = 0
    join_str = " "
    Len = len(words)
    if Len > 1:
        lst = []
        for i in range(Len-1):
            for k in range(1,skip+2):
                if i+k < Len:
                    lst.append(join_str.join([words[i],words[i+k]]))
    else:
        #set it as unigram
        lst = create_unigram(words)
    return lst

def create_trigrams(words):
    assert type(words) == list
    skip = 0
    join_str = " "
    Len = len(words)
    if Len > 2:
        lst = []
        for i in range(1,skip+2):
            for k1 in range(1, skip+2):
                for k2 in range(1,skip+2):
                    if i+k1 < Len and i+k1+k2 < Len:
                        lst.append(join_str.join([words[i], words[i+k1],words[i+k1+k2]]))
                      
        else:
           lst = create_bigrams(words)
        return lst
 




porter = PorterStemmer()

def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]





In [22]:
#create tf-df frequency features
#tf-idf 
countV = CountVectorizer()
train_count = countV.fit_transform(train_news['article'].values)
tfidfV = TfidfTransformer()
train_tfidf = tfidfV.fit_transform(train_count)

def get_tfidf_stats():
    train_tfidf.shape
    #get train data feature names 
    print(train_tfidf.A[:10])




In [23]:



#create tf-df frequency features
#tf-idf 
countV = CountVectorizer()
train_count = countV.fit_transform(train_news['article'].values)
tfidfV = TfidfTransformer()
train_tfidf = tfidfV.fit_transform(train_count)

def get_tfidf_stats():
    train_tfidf.shape
    #get train data feature names 
    print(train_tfidf.A[:10])


In [24]:


from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score




def build_confusion_matrix(classifier):
    
    k_fold = KFold(n_splits=5)
    scores = []
    confusion = np.array([[0,0],[0,0]])

    for train_ind, test_ind in k_fold.split(train_news):
        train_text = train_news.iloc[train_ind]['article'] 
        train_y = train_news.iloc[train_ind]['label']
    
        test_text = train_news.iloc[test_ind]['article']
        test_y = train_news.iloc[test_ind]['label']
        
        classifier.fit(train_text,train_y)
        predictions = classifier.predict(test_text)
        
        confusion += confusion_matrix(test_y,predictions)
        score = f1_score(test_y,predictions)
        scores.append(score)
    
    return (print('Total statements classified:', len(train_news)),
    print('Score:', sum(scores)/len(scores)),
    print('score length', len(scores)))
   
    


tfidf_ngram = TfidfVectorizer(stop_words=set(adv.stopwords['tagalog']),ngram_range=(1,3),use_idf=True,smooth_idf=True)


#linear SVM classifier
svm_pipeline_ngram = Pipeline([
        ('svm_tfidf',tfidf_ngram),
        ('svm_clf',SVC(probability = True))
        ])


svm_pipeline_ngram.fit(train_news['article'],train_news['label'])
predicted_svm_ngram = svm_pipeline_ngram.predict(test_news['article'])
np.mean(predicted_svm_ngram == test_news['label'])


build_confusion_matrix(svm_pipeline_ngram)


model_file = 'final_model_fil.sav'



KeyboardInterrupt: 

In [ ]:
import pickle
var = "Ayon sa TheWrap.com, naghain ng kaso si Krupa, 35, noong Huwebes dahil nakaranas umano siya ng emotional distress bunga ng mga malisyosong pahayag ni Glanville, hindi lamang tungkol sa maselang bahagi ng kanyang katawan kundi pati na rin sa kanyang buhay pag-ibig. Noong Nobyembre 2013, napanood si Brandi sa Watch What Happens Live at sinabing kinausap umano ito ng dating karelasyon ni Joanna. Si Mohamed Hadid, na noon ay kasal sa cast member ng Real Housewives of Beverly Hills na si Yolanda Foster habang nasa relasyon, ang nagsabing ang kanyang ""p--sy smelled."" (Classy pa rin naman ang pagkakabanggit dito ni Brandi.) Noong Disyembre, gumawa ng mga panibagong komento si Glanville, 42, sa kanyang pagharap sa YouTube at sinabing, ""You can't help the odor situation."" Makalipas ang ilang linggo, nagpadala ng sulat ang mga abogado ni Krupa kay Brandi tungkol sa kanilang planong pagsampa ng kaso. Nakalakip sa sulat ng abogado ni Joanna na, ""You knew your statements were false and that such were damaging and humiliating to Mrs. Krupa and/or her husband and subjected Mrs. Krupa... to hatred, ridicule, contempt and/or injury."" ""Moreover, it appears you make false statements about Mrs. Krupa to make yourself marketable and to associate your name with a celebrity in attempt to 'tag' yourself with her for media and personal searches and/or to create a brand for yourself for monetary and attempted celebrity status,"" parte pa rin ng sulat. ""Your statements are malicious and knowingly false."" Laman ng kaso ang umano'y danyos sa pagtitiis ni Krupa na mas mataas pa sa $15,000."
#pickle.dump(svm_pipeline_ngram,open(model_file,'wb')) 
load_model = pickle.load(open('final_model_fil.sav', 'rb'))
prediction = load_model.predict([var])
prob = load_model.predict_proba([var])


print("The given statement is ",prediction[0])
print("The probability score is ",prob[0][1])





The given statement is  False
The probability score is  5.8409663768844545e-06
